In [3]:
#from nba_api.stats.static import players
#
#
# players.find_players_by_full_name("Lebron James")
#
#players.find_player_by_id(2544)

In [1]:
#Module load
from nba_api.stats.endpoints import commonallplayers #per avere l'elenco dei giocatori (id)
from nba_api.stats.endpoints import playercareerstats #per avere i dati
from nba_api.stats.endpoints import playerawards #per sapere chi ha vinto MVP
from nba_api.stats.endpoints import TeamYearByYearStats #classifiche finali delle squadre
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
from_year = 2010

total_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]
total_players['TO_YEAR'] = total_players['TO_YEAR'].astype('int')
total_players = total_players[total_players['TO_YEAR'] > from_year]

#Ho controllato che in total_players, il PERSON_ID sia univoco
total_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
19,1629121,"Adams, Jaylen",Jaylen Adams,0,2018,2020,jaylen_adams,jaylen_adams,0,,,,,None,Y,01
20,203919,"Adams, Jordan",Jordan Adams,0,2014,2015,jordan_adams,jordan_adams,0,,,,,None,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,203092,"Zeller, Tyler",Tyler Zeller,0,2012,2019,tyler_zeller,tyler_zeller,0,,,,,None,Y,00
4720,1627757,"Zimmerman, Stephen",Stephen Zimmerman,0,2016,2016,stephen_zimmerman,stephen_zimmerman,0,,,,,None,Y,01
4721,1627835,"Zipser, Paul",Paul Zipser,0,2016,2017,paul_zipser,paul_zipser,0,,,,,None,Y,01
4722,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01


In [43]:


total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
total_stats = pd.DataFrame()


for index, row in tqdm(total_names.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]
    stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)
    #stats = stats[stats[]] PRENDERE SOLO I DATI SUPERIORE ALL'ANNO FROM_YEAR
    #name = row[["DISPLAY_FIRST_LAST"]].repeat(stats.shape[0])
    #tot = pd.concat([stats, name], axis=1, ignore_index=True)
    awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]
    awards = awards.drop(awards.columns[[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1)
    awards = awards.rename(columns={"SEASON": "SEASON_ID"})
    awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"})
    #awards = awards.drop(awards.row()) #togliere tutti quelli != da MVP


    total_stats = pd.concat([total_stats, stats, awards],axis=0,ignore_index=True)

#Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])]
#Prendo i dati superiore all'anno FROM_YEAR
total_stats= total_stats[total_stats["SEASON_ID"] > from_year]

with open("player_stats.pkl",'wb') as f:
    pickle.dump(total_stats,f)

#Find lebron james totals stats
#playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]

7it [02:50, 24.37s/it]


KeyboardInterrupt: 

#How it's structured this dataset
- Player_id --> id del giocatore(da ritrovare il nome con players.find_player_by_id())
- Season_id --> id della stagione
- ...other...
- GP --> Games Played
- GS --> Games ... (idk but similar to GP) (I think is not usefull)
- MIN Minutes Played
- FGM Field Goals Made --> Numero canestri fatti
- FGA Field Goals Attempted --> Numero tiri provati
- FG% Field Goal Percentage --> FGM/FGA, rapporto precisione tiri
- 3PM 3 Point Field Goals Made --> Numero canestri da 3 fatti
- 3PA 3 Point Field Goals Attempted --> Numero tiri da 3 provati
- 3P% 3 Point Field Goals Percentage --> 3PM/3PA, precisione tiri da 3
- FTM Free Throws Made --> Tiri liberi fatti
- FTA Free Throws Attempted --> tiri liberi provati
- FT% Free Throw Percentage --> FTM/FTA
- OREB Offensive Rebounds --> Rimbalzi conquistati in attacco
- DREB Defensive Rebounds --> rimbalzi conquistati in difesa
- REB Rebounds --> OREB + DREB, rimbalzi totali
- AST Assists
- TOV Turnovers --> palla persa
- STL Steals --> palla rubata
- BLK Blocks --> blocchi
- PF Personal Fouls --> falli personali
- DD2 Double Doubles --> doppie doppie
- TD3 Trible Doubles --> triple doppie
- PTS Points

In [36]:
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)




In [37]:

total_stats


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203518,2017,00,1610612760,OKC,23.0,68,6,1055.0,134,...,0.898,18,68,86,40,37,8,33,114,406
1,203518,2018,00,1610612760,OKC,24.0,75,8,1134.0,115,...,0.848,26,88,114,28,38,8,25,124,353
2,203518,2019,00,1610612760,OKC,25.0,31,2,588.0,56,...,0.923,5,43,48,20,17,6,14,53,165
3,1630173,2021,00,1610612748,MIA,21.0,61,4,737.0,124,...,0.509,73,135,208,29,20,28,43,91,304
4,1630173,2022,00,1610612761,TOR,22.0,25,17,625.0,83,...,0.574,56,134,190,37,13,14,30,56,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,1627826,2019,00,1610612746,LAC,22.0,26,25,524.0,100,...,0.733,61,139,200,38,10,24,37,64,244
9775,1627826,2019,00,0,TOT,22.0,59,37,1039.0,212,...,0.802,115,247,362,63,14,51,70,137,525
9776,1627826,2020,00,1610612746,LAC,23.0,72,70,1326.0,236,...,0.747,197,346,543,82,16,66,61,168,596
9777,1627826,2021,00,1610612746,LAC,24.0,72,33,1609.0,257,...,0.789,189,330,519,90,24,62,81,187,650


In [4]:
from nba_api.stats.endpoints import teamyearbyyearstats

teamyearbyyearstats.TeamYearByYearStats(team_id=1610612748).get_data_frames()[0]

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612748,Miami,Heat,1988-89,82,15,67,0.183,13,6,...,1309,2211,3520,1958,2124,744,1728,408,8016,25
1,1610612748,Miami,Heat,1989-90,82,18,64,0.220,11,5,...,1242,2313,3555,1957,2123,736,1557,388,8247,24
2,1610612748,Miami,Heat,1990-91,82,24,58,0.293,13,6,...,1232,2302,3534,1904,2080,756,1551,387,8349,21
3,1610612748,Miami,Heat,1991-92,82,38,44,0.463,8,4,...,1187,2366,3553,1749,1819,670,1377,373,8608,14
4,1610612748,Miami,Heat,1992-93,82,36,46,0.439,11,5,...,1134,2384,3518,1688,2011,609,1287,350,8495,20
5,1610612748,Miami,Heat,1993-94,82,42,40,0.512,8,4,...,1235,2407,3642,1856,2024,643,1315,374,8475,7
6,1610612748,Miami,Heat,1994-95,82,32,50,0.390,10,4,...,1092,2272,3364,1779,2000,662,1291,298,8293,14
7,1610612748,Miami,Heat,1995-96,82,42,40,0.512,8,3,...,999,2495,3494,1752,2158,574,1394,439,7909,23
8,1610612748,Miami,Heat,1996-97,82,61,21,0.744,2,1,...,957,2402,3359,1735,1919,650,1306,439,7776,22
9,1610612748,Miami,Heat,1997-98,82,55,27,0.671,2,1,...,1028,2419,3447,1758,1982,665,1226,429,7787,16


In [9]:
from nba_api.stats.endpoints import playerawards

playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]

,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2008-09,None,None,1610612739,Award,Kia Motors,KIADT,None
1,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2009-10,None,None,1610612739,Award,Kia Motors,KIADT,None
2,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2010-11,None,None,1610612748,Award,Kia Motors,KIADT,None
3,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2011-12,None,None,1610612748,Award,Kia Motors,KIADT,None
4,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2012-13,None,None,1610612748,Award,Kia Motors,KIADT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Most Valuable Player of the ...,None,2005-06,None,None,None,Award,Sporting News,SNMVP,None
145,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Rookie of the Year,None,2003-04,None,None,None,Award,Sporting News,SNROY,None
146,2544,LeBron,James,USA,Olympic Bronze Medal,None,2004,None,None,None,Award,Olympic,Bronze,None
147,2544,LeBron,James,USA,Olympic Gold Medal,None,2008,None,None,None,Award,Olympic,Gold,None
